In [1]:
%load_ext lab_black
%cd ../../..

/mnt/h/hev/log-analytics


In [17]:
from tqdm import tqdm
import torch, sys
import numpy as np
import pandas as pd
import pickle

In [3]:
df = pd.read_csv("data/ori/test.csv")

In [4]:
deck1 = torch.load("results/distilbert-base-uncased-ver7/distilbert-base-uncased-focal-AdamW-lr1e-05-ver7-os10_3-deck1.pth")
deck2 = torch.load("results/distilbert-base-uncased-ver7/distilbert-base-uncased-focal-AdamW-lr1e-05-ver7-os10_3-deck2.pth")

In [5]:
deck1["feat"].shape, deck2["feat"].shape

(torch.Size([421079, 768]), torch.Size([1095951, 768]))

In [6]:
deck1["feat_"] = deck1["feat"].cuda()
deck2["feat_"] = deck2["feat"].cuda()

In [7]:
deck1.keys(), deck2.keys()

(dict_keys(['feat', 'otext', 'tlevel', 'fclevel', 'feat_']),
 dict_keys(['feat', 'otext', 'fclevel', 'ids', 'feat_']))

In [10]:
j = 100000
shown = False

In [9]:
dist_ = torch.norm(deck1["feat_"] - deck2["feat_"][j, None], p=None, dim=1)
values_, indices_ = dist_.topk(4, largest=False)
tlevels = deck1["tlevel"][indices_]
print("values:", values_)
print("indices:", indices_)
print("id:", deck1["tlevel"][indices_])
j += 1

values: tensor([0.0031, 0.0031, 0.0031, 0.0031], device='cuda:0')
indices: tensor([17556, 17365, 17671, 19175], device='cuda:0')
id: tensor([0, 0, 0, 0])


In [25]:
fps = [
    open(f"results/distilbert-base-uncased-ver7/distilbert-base-uncased-focal-AdamW-lr1e-05-ver7-os10_3-test_lv{i}.txt", "a")
    for i in range(7)
] + [open(f"results/distilbert-base-uncased-ver7/distilbert-base-uncased-focal-AdamW-lr1e-05-ver7-os10_3-test_lvrd.txt", "a")]

In [26]:
with tqdm(total=100000, ncols=100, file=sys.stdout) as t:
    for i in range(100000, 200000):
        dist_ = torch.norm(deck1["feat_"] - deck2["feat_"][i, None], p=None, dim=1)
        values_, indices_ = dist_.topk(4, largest=False)
        tlevels = deck1["tlevel"][indices_]

        if not (deck1["tlevel"][indices_[0]] in [0, 1, 3, 5] and values_[0] < 0.5):
            lv = 0
            while lv < 7:
                if (tlevels == lv).all():
                    break
                lv += 1
            fp = fps[lv]
            fp.write("*  Submit : " + df.full_log[deck2["ids"][i - 1][0] - 1000000])
            fp.write("----------------------------------------------------------------------------------------\n")
            fp.write("*   index : " + str(i - 1) + "\n")
            fp.write("*    dist : " + " ".join(map(lambda x: f"{x:.4f}", values_.cpu().tolist())) + "\n")
            fp.write("* fc-level: " + str(deck2["fclevel"][i - 1]) + "\n")
            fp.write("----------------------------------------------------------------------------------------\n")

            for idx in indices_:
                fp.write("* Level " + str(deck1["tlevel"][idx].item()) + " : " + deck1["otext"][idx] + "\n")
                fp.write("----------------------------------------------------------------------------------------\n")

            fp.write("\n\n")
            fp.flush()

        t.update()

100%|██████████████████████████████████████████████████████| 100000/100000 [12:03<00:00, 138.29it/s]


In [11]:
while shown or (deck1["tlevel"][indices_[0]] in [0, 1, 3, 5] and values_[0] < 0.5):
    shown = False

    dist_ = torch.norm(deck1["feat_"] - deck2["feat_"][j, None], p=None, dim=1)
    values_, indices_ = dist_.topk(4, largest=False)
    tlevels = deck1["tlevel"][indices_]
    j += 1

In [12]:
print("*  Submit :", df.full_log[deck2["ids"][j - 1][0] - 1000000])
print("----------------------------------------------------------------------------------------")
print("*   index :", j - 1)
print("*    dist :", " ".join(map(lambda x: f"{x:.4f}", values_.cpu().tolist())))
print("* fc-level:", deck2["fclevel"][j - 1])
print("----------------------------------------------------------------------------------------")

for idx in indices_:
    print("* Level", deck1["tlevel"][idx].item(), ":", deck1["otext"][idx])
    print("----------------------------------------------------------------------------------------")

shown = True

*  Submit : Jan 25 16:50:01 localhost kernel: Out of memory: Kill process 11235 (wazuh-modulesd) score 268 or sacrifice child
----------------------------------------------------------------------------------------
*   index : 107532
*    dist : 6.3645 6.5072 7.9409 7.9421
* fc-level: tensor(0)
----------------------------------------------------------------------------------------
* Level 3 : suricata: [1:2010494:4] ET SCAN Multiple MySQL Login Failures Possible Brute Force Attempt [Classification: Attempted Information Leak] [Priority: 2] {TCP} 192.168.0.42:3306 -> 192.168.0.160:35188
----------------------------------------------------------------------------------------
* Level 0 : suricata: [1:2008453:7] ET SCAN Tomcat Auth Brute Force attempt (admin) [Classification: Web Application Attack] [Priority: 1] {TCP} 192.168.0.11:58311 -> 192.168.0.160:55000
----------------------------------------------------------------------------------------
* Level 0 : juniper
---------------------

---

4개가 다 일치할 때는 level별로 threshold를 따로 두는 방식으로 하고
* level0 - 좀 낮게 (비교군이 많기 때문에) - 0.8?
* level1 - 전처리를 없애니까 dist가 증가했음. 0.8?
* level2 - 
* level3 - 꽤 높게 (변화무쌍하니까) - 1.0?
* level4 - 
* level5 - 꽤 높게 (변화무쌍하니까) - 1.0
* level6 - 나름 변화가 큼 - 0.8
* 섞인 경우 - 비슷한 케이스가 별로 없었다는 뜻이니까 낮게 - 0.2

2, 4, 6에 대해서는 아직 데이터가 더 모여야 하겠다

level0 길이가 긴거는 맞아도 dist가 높게(0.3이상) 나오고, 길이가 짧은건 틀려도 dist가 탖게 나오는 것 같은데...  
길이도 입력 인수로 해야할까?

※ TODO: 전체에 대해 dist 구한 다음에 각 level/군집별로 top dist 분포가 어떻게 나타나는지 그래프를 그려보자